In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import pymysql as mdb
%matplotlib inline
import geocoder
import seaborn as sns
from bs4 import BeautifulSoup
import urllib 
import pickle
import pymysql as mdb

/usr/local/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [2]:
#need to pickle after each pool so i don't have to rerun if interrupted

## Pool schedules have changed. I need to rescrape schedule for each pool. This time I will note website quirks of each pool for furture updates.

In [2]:
pools = pd.read_csv('pools.csv')
pools.head()

,swimming_pool,website_source,address
0,Asser Levy,http://www.nycgovparks.org/facilities/recreati...,"392 asser levy pl, NY"
1,Chelsea,http://www.nycgovparks.org/facilities/recreati...,"430 W 25th St, New York, NY 10001"
2,Gertrude Eredel,http://www.nycgovparks.org/facilities/recreati...,"232 West 60th Street, ny"
3,Hansborough,http://www.nycgovparks.org/facilities/recreati...,"35 west 134th street, NY"
4,Tony Dapolito,http://www.nycgovparks.org/facilities/recreati...,"1 clarkson st., NY"


In [4]:
poolsdict = pickle.load(open("poolsandcoords.p", "rb"))

In [3]:
days_of_week = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

In [4]:
urls = pools['website_source'].tolist()

In [206]:
poolist = pools['swimming_pool'].tolist()
poolist

['Asser Levy',
 'Chelsea',
 'Gertrude Eredel',
 'Hansborough',
 'Tony Dapolito',
 'Brownsville',
 'Metropolitan',
 'St. Johns',
 'Flushing Meadows Corona Park',
 'Roy Wilkins',
 'Saint Marys',
 'Recreation Center 54']

In [8]:
#check former database to be consistent with naming
con = mdb.connect('localhost', 'root', '', 'lap_schedule')
    
schedule = pd.read_sql('SELECT * FROM lap_schedule_table5' , con)
pooltable = pd.read_sql('SELECT * FROM  pool_table6'  , con)

In [9]:
pooltable.head()

,index,swimming_pool,address
0,0,Asser Levy,"392 asser levy place, Manhattan, NY"
1,1,Chelsea,"430 west 25th street, Manhattan, NY"
2,2,Gertrude Eredel,"232 west 60th street, Manhattan, ny"
3,3,Hansborough,"35 west 134th street, Manhattan NY"
4,4,Tony Dapolito,"1 clarkson street, Manhattan, NY"


## go pool by pool and create a dataframe noting differences in website notations

In [122]:
#start with asser_levy
r = urllib.urlopen(urls[0]).read()
soup = BeautifulSoup(r, "lxml")
other = ["program", "center-hrs"]
letters = soup.find_all(["p", "div"], other ) 
swimposting = []
otherposting = []
for i in letters:
    if str(i).find('Adult Lap Swim') > 0:
        swimposting.append(str(i))
    if str(i).find('Building Hours') > 0:
        swimposting.append(str(i))
    else:
        otherposting.append(str(i))
asserlevy = swimposting[7:]
#make a loop that saves lap swim programs and substitutes the days of the week for the building hours which separate them
asser_levy_sched = {}
count = 0
for i in asserlevy:
    if str(i).find('Building Hours') > 0:
        asser_levy_sched[days_of_week[count]] = []
        count = count + 1
    if str(i).find('Adult Lap Swim') > 0:
        asser_levy_sched[days_of_week[count-1]].append(re.findall('[0-9]*:[0-9]*\s[a-z]\s-\s[0-9]*:[0-9]*\s[a-z]'  ,str(i)))

In [123]:
asser_levy_sched

{'Friday': [['7:00 a - 9:00 a'], ['1:15 p - 3:00 p'], ['7:00 p - 9:20 p']],
 'Monday': [['7:00 a - 9:00 a'], ['1:15 p - 3:45 p'], ['6:30 p - 9:20 p']],
 'Saturday': [['8:00 a - 9:00 a'], ['2:15 p - 4:30 p']],
 'Sunday': [['8:00 a - 11:00 a'], ['2:15 p - 4:30 p']],
 'Thursday': [['7:00 a - 9:00 a'], ['1:15 p - 3:30 p'], ['6:30 p - 9:20 p']],
 'Tuesday': [['7:00 a - 9:00 a'], ['1:15 p - 3:45 p'], ['6:30 p - 9:20 p']],
 'Wednesday': [['7:00 a - 9:00 a'], ['1:15 p - 3:45 p'], ['6:30 p - 9:20 p']]}

In [124]:
#i can pickle this
pickle.dump( asser_levy_sched, open("asser_levy_sched.p", "wb" ) )

In [128]:
#now on to chelsea
r = urllib.urlopen(urls[1]).read()
soup = BeautifulSoup(r, "lxml")
other = ["program", "center-hrs"]
letters = soup.find_all(["p", "div"], other ) 


In [141]:
#laps are denoted as Lap Swim on the chelsea page
swimposting = []
otherposting = []
for i in letters:
    if str(i).find('Lap Swim') > 0:
        swimposting.append(str(i))
    if str(i).find('Building Hours') > 0:
        swimposting.append(str(i))
    else:
        otherposting.append(str(i))


In [142]:
chelsea = swimposting[7:-7] #sometimes more building hours after swim schedule
#make a loop that saves lap swim programs and substitutes the days of the week for the building hours which separate them
chelsea_sched = {}
count = 0

for i in chelsea:
    if str(i).find('Building Hours') > 0:
        chelsea_sched[days_of_week[count]] = []
        count = count + 1
    if str(i).find('Lap Swim') > 0:
        chelsea_sched[days_of_week[count-1]].append(re.findall('[0-9]*:[0-9]*\s[a-z]\s-\s[0-9]*:[0-9]*\s[a-z]'  ,str(i)))

In [143]:
chelsea_sched

{'Friday': [['7:00 a - 9:30 a'], ['2:00 p - 3:45 p'], ['9:00 p - 10:00 p']],
 'Monday': [['7:00 a - 10:00 a'], ['2:00 p - 3:45 p'], ['9:00 p - 10:00 p']],
 'Saturday': [['7:00 a - 8:00 a']],
 'Sunday': [['8:00 a - 11:00 a'], ['3:00 p - 4:00 p']],
 'Thursday': [['7:00 a - 10:00 a'],
  ['1:00 p - 3:45 p'],
  ['6:00 p - 7:00 p'],
  ['9:00 p - 10:00 p']],
 'Tuesday': [['7:00 a - 10:00 a'], ['1:00 p - 3:45 p'], ['6:00 p - 7:00 p']],
 'Wednesday': [['7:00 a - 9:30 a'], ['2:00 p - 3:45 p'], ['9:00 p - 10:00 p']]}

In [144]:
#i can pickle this
pickle.dump( chelsea_sched, open("chelsea_sched.p", "wb" ) )

In [156]:
#next pool is gertrude eredel which denotes laps as Lap Swim
r = urllib.urlopen(urls[2]).read()
soup = BeautifulSoup(r, "lxml")
other = ["program", "center-hrs"]
letters = soup.find_all(["p", "div"], other ) 

swimposting = []
otherposting = []
for i in letters:
    if str(i).find('Lap Swim') > 0:
        swimposting.append(str(i))
    if str(i).find('Building Hours') > 0:
        swimposting.append(str(i))
    else:
        otherposting.append(str(i))

gertrude = swimposting[7:-7] 
gertrude_sched = {}
count = 0
for i in gertrude:
    if str(i).find('Building Hours') > 0:
        gertrude_sched[days_of_week[count]] = []
        count = count + 1
    if str(i).find('Lap Swim') > 0:
        gertrude_sched[days_of_week[count-1]].append(re.findall('[0-9]*:[0-9]*\s[a-z]\s-\s[0-9]*:[0-9]*\s[a-z]'  ,str(i)))


In [157]:
gertrude_sched

{'Friday': [['7:00 a - 9:45 a'], ['1:00 p - 3:45 p'], ['7:15 p - 9:00 p']],
 'Monday': [['7:00 a - 9:45 a'], ['6:15 p - 9:00 p']],
 'Saturday': [['8:00 a - 9:45 a']],
 'Sunday': [],
 'Thursday': [['7:00 a - 9:45 a'], ['12:00 p - 2:00 p'], ['7:15 p - 9:00 p']],
 'Tuesday': [['7:00 a - 9:45 a'], ['12:00 p - 2:00 p'], ['6:15 p - 9:00 p']],
 'Wednesday': [['7:00 a - 9:45 a'], ['6:15 p - 9:00 p']]}

In [158]:
#i can pickle this
pickle.dump( gertrude_sched, open("gertrude_sched.p", "wb" ) )

In [161]:
#on to hansborough which denotes as Adult Lap Swim
r = urllib.urlopen(urls[3]).read()
soup = BeautifulSoup(r, "lxml")
other = ["program", "center-hrs"]
letters = soup.find_all(["p", "div"], other ) 

swimposting = []
otherposting = []
for i in letters:
    if str(i).find('Adult Lap Swim') > 0:
        swimposting.append(str(i))
    if str(i).find('Building Hours') > 0:
        swimposting.append(str(i))
    else:
        otherposting.append(str(i))

hansborough = swimposting[7:-7] 
hansborough_sched = {}
count = 0
for i in hansborough:
    if str(i).find('Building Hours') > 0:
        hansborough_sched[days_of_week[count]] = []
        count = count + 1
    if str(i).find('Adult Lap Swim') > 0:
        hansborough_sched[days_of_week[count-1]].append(re.findall('[0-9]*:[0-9]*\s[a-z]\s-\s[0-9]*:[0-9]*\s[a-z]'  ,str(i)))


In [162]:
hansborough_sched

{'Friday': [['7:00 a - 8:45 a'], ['3:00 p - 4:15 p'], ['7:15 p - 9:00 p']],
 'Monday': [['7:00 a - 8:45 a'], ['1:00 p - 3:00 p'], ['7:15 p - 9:00 p']],
 'Saturday': [['9:00 a - 10:15 a'], ['3:00 p - 4:00 p']],
 'Sunday': [],
 'Thursday': [['9:00 a - 10:00 a'], ['1:15 p - 3:45 p'], ['7:15 p - 9:00 p']],
 'Tuesday': [['9:00 a - 10:00 a'], ['1:15 p - 3:45 p'], ['7:15 p - 9:00 p']],
 'Wednesday': [['7:00 a - 8:45 a'], ['2:00 p - 3:00 p'], ['7:15 p - 9:00 p']]}

In [163]:
#i can pickle this
pickle.dump( hansborough_sched, open("hansborough_sched.p", "wb" ) )


In [201]:
#on to tony dapolito denotes by Adult Lap Swim
r = urllib.urlopen(urls[4]).read()
soup = BeautifulSoup(r, "lxml")
other = ["program", "center-hrs"]
letters = soup.find_all(["p", "div"], other ) 


swimposting = []
otherposting = []
for i in letters:
    if str(i).find('Adult Lap Swim') > 0:
        swimposting.append(str(i))
    if str(i).find('Building Hours') > 0:
        swimposting.append(str(i))
    else:
        otherposting.append(str(i))
tony_dapolito = swimposting[7:-7]
#make a loop that saves lap swim programs and substitutes the days of the week for the building hours which separate them
tony_dapolito_sched = {}
count = 0
for i in tony_dapolito:
    if str(i).find('Building Hours') > 0:
        tony_dapolito_sched[days_of_week[count]] = []
        count = count + 1
    if str(i).find('Adult Lap Swim') > 0:
        tony_dapolito_sched[days_of_week[count-1]].append(re.findall('[0-9]*:[0-9]*\s[a-z]\s-\s[0-9]*:[0-9]*\s[a-z]'  ,str(i)))

In [202]:
tony_dapolito_sched

{'Friday': [['7:00 a - 9:30 a'], ['11:30 a - 2:15 p'], ['5:00 p - 7:15 p']],
 'Monday': [['7:00 a - 9:30 a'], ['11:30 a - 2:15 p'], ['4:00 p - 9:30 p']],
 'Saturday': [['3:15 p - 4:30 p']],
 'Sunday': [['9:00 a - 10:30 a'], ['3:15 p - 4:30 p']],
 'Thursday': [['7:00 a - 8:30 a'], ['12:15 p - 2:15 p'], ['8:00 p - 9:30 p']],
 'Tuesday': [['7:00 a - 8:30 a'], ['12:15 p - 2:15 p'], ['8:00 p - 9:30 p']],
 'Wednesday': [['7:00 a - 9:30 a'],
  ['11:30 a - 2:15 p'],
  ['4:00 p - 6:00 p'],
  ['7:15 p - 9:30 p']]}

In [203]:
#i can pickle this
pickle.dump( tony_dapolito_sched, open("tony_dapolito_sched.p", "wb" ) )

In [215]:
#poolsched = ['Browsnvillesched', 'Metropolitansched' , 'St_Johnssched','Recreation_Center_54sched']
#poolsadultnum = [5,6, 7, 11 ]
r = urllib.urlopen(urls[5]).read()
soup = BeautifulSoup(r, "lxml")
other = ["program", "center-hrs"]
letters = soup.find_all(["p", "div"], other ) 

swimposting = []
otherposting = []
for i in letters:
    if str(i).find('Adult Lap Swim') > 0:
        swimposting.append(str(i))
    if str(i).find('Building Hours') > 0:
        swimposting.append(str(i))
    else:
        otherposting.append(str(i))

brownsville = swimposting[7:-7] 
brownsville_sched = {}
count = 0
for i in brownsville:
    if str(i).find('Building Hours') > 0:
        brownsville_sched[days_of_week[count]] = []
        count = count + 1
    if str(i).find('Adult Lap Swim') > 0:
        brownsville_sched[days_of_week[count-1]].append(re.findall('[0-9]*:[0-9]*\s[a-z]\s-\s[0-9]*:[0-9]*\s[a-z]'  ,str(i)))
pickle.dump( brownsville_sched, open("brownsville_sched.p", "wb" ) )

In [214]:
brownsville_sched

{'Friday': [['2:15 p - 3:00 p'], ['5:15 p - 7:00 p']],
 'Monday': [['2:15 p - 3:00 p'], ['5:15 p - 7:00 p']],
 'Saturday': [],
 'Sunday': [],
 'Thursday': [],
 'Tuesday': [],
 'Wednesday': [['2:15 p - 3:00 p'], ['5:15 p - 7:00 p']]}

In [216]:
#AHHHHHH there are scraping issues, pool hours in the dataframe below are not correct. retry metropolitan
r = urllib.urlopen(urls[6]).read()
soup = BeautifulSoup(r, "lxml")
other = ["program", "center-hrs"]
letters = soup.find_all(["p", "div"], other )
swimposting = []
otherposting = []
for i in letters:
    if str(i).find('Adult Lap Swim') > 0:
        swimposting.append(str(i))
    if str(i).find('Building Hours') > 0:
        swimposting.append(str(i))
    else:
        otherposting.append(str(i))

metropolitan = swimposting[7:]
metropolitan_sched = {}
count = 0
for i in metropolitan:
    if str(i).find('Building Hours') > 0:
        metropolitan_sched[days_of_week[count]] = []
        count = count + 1
    if str(i).find('Adult Lap Swim') > 0:
        metropolitan_sched[days_of_week[count-1]].append(re.findall('[0-9]*:[0-9]*\s[a-z]\s-\s[0-9]*:[0-9]*\s[a-z]'  ,str(i)))

In [219]:
pickle.dump( metropolitan_sched, open("metropolitan_sched.p", "wb" ) )

In [221]:
#poolsched = ['Browsnvillesched', 'Metropolitansched' , 'St_Johnssched','Recreation_Center_54sched']
#poolsadultnum = [5,6, 7, 11 ]
r = urllib.urlopen(urls[7]).read()
soup = BeautifulSoup(r, "lxml")
other = ["program", "center-hrs"]
letters = soup.find_all(["p", "div"], other ) 

swimposting = []
otherposting = []
for i in letters:
    if str(i).find('Adult Lap Swim') > 0:
        swimposting.append(str(i))
    if str(i).find('Building Hours') > 0:
        swimposting.append(str(i))
    else:
        otherposting.append(str(i))

stjohns = swimposting[7:] 
stjohns_sched = {}
count = 0
for i in stjohns:
    if count ==7:
        break
    else:
        if str(i).find('Building Hours') > 0:
            stjohns_sched[days_of_week[count]] = []
            count = count + 1
        if str(i).find('Adult Lap Swim') > 0:
            stjohns_sched[days_of_week[count-1]].append(re.findall('[0-9]*:[0-9]*\s[a-z]\s-\s[0-9]*:[0-9]*\s[a-z]'  ,str(i)))
pickle.dump( stjohns_sched, open("stjohns_sched.p", "wb" ) )

In [223]:
urls[7]

'http://www.nycgovparks.org/facilities/recreationcenters/B245/schedule#Pool'

In [235]:
#poolsched = ['Browsnvillesched', 'Metropolitansched' , 'St_Johnssched','Recreation_Center_54sched']
#poolsadultnum = [5,6, 7, 11 ]
r = urllib.urlopen(urls[11]).read()
soup = BeautifulSoup(r, "lxml")
other = ["program", "center-hrs"]
letters = soup.find_all(["p", "div"], other ) 

swimposting = []
otherposting = []
for i in letters:
    if str(i).find('Adult Lap Swim') > 0:
        swimposting.append(str(i))
    if str(i).find('Building Hours') > 0:
        swimposting.append(str(i))
    else:
        otherposting.append(str(i))

reccenter54 = swimposting[7:] 
reccenter54_sched = {}
count = 0
for i in reccenter54:
    if count == 8:
        break
    else:
        if str(i).find('Building Hours') > 0:
            reccenter54_sched[days_of_week[count]] = []
            count = count + 1
        if str(i).find('Adult Lap Swim') > 0:
            reccenter54_sched[days_of_week[count-1]].append(re.findall('[0-9]*:[0-9]*\s[a-z]\s-\s[0-9]*:[0-9]*\s[a-z]'  ,str(i)))
pickle.dump( reccenter54_sched, open("reccenter54_sched.p", "wb" ) )

IndexError: list index out of range

In [227]:
reccenter54_sched

{'Friday': [['7:00 a - 9:45 a'], ['12:15 p - 3:00 p'], ['7:15 p - 9:00 p']],
 'Monday': [['7:00 a - 9:45 a'], ['12:15 p - 3:00 p'], ['7:15 p - 9:00 p']],
 'Saturday': [['8:00 a - 10:15 a']],
 'Sunday': [],
 'Thursday': [['7:00 a - 9:45 a'], ['12:15 p - 3:00 p'], ['7:45 p - 9:00 p']],
 'Tuesday': [['7:00 a - 9:45 a'], ['12:15 p - 3:00 p'], ['7:45 p - 9:00 p']],
 'Wednesday': [['7:00 a - 9:45 a'], ['12:15 p - 3:00 p'], ['7:15 p - 9:00 p']]}

In [16]:
#make a loop for all pools with same denotation of Adult Lap Swim this isn't correct yet
poolsadult = ['Browsnville', 'Metropolitan' , 'St_Johns','Recreation_Center_54']
poolsadultdict = ['Browsnville', 'Metropolitan' , 'St_Johns','Recreation_Center_54']
poolsched = ['Browsnvillesched', 'Metropolitansched' , 'St_Johnssched','Recreation_Center_54sched']
poolsched2 = ['Browsnvillesched', 'Metropolitansched' , 'St_Johnssched','Recreation_Center_54sched']
poolsadultnum = [5,6, 7, 11 ]

pickle.dump( h, open("%s.p" %(poolsched2[counter]), "wb" ) )
                      

In [233]:

#8, Flushing Meadows Corona Park is just Lap Swim
#9, Roy Wilkins is Adult and Senior Swim
#10, Saint Marys is Adult Swim

r = urllib.urlopen(urls[8]).read()
soup = BeautifulSoup(r, "lxml")
other = ["program", "center-hrs"]
letters = soup.find_all(["p", "div"], other ) 

swimposting = []
otherposting = []
for i in letters:
    if str(i).find('Lap Swim') > 0:
        swimposting.append(str(i))
    if str(i).find('Building Hours') > 0:
        swimposting.append(str(i))
    else:
        otherposting.append(str(i))
flushing = swimposting[7:] 
flushing_sched = {}
count = 0
for i in flushing:
    if count == 8:
        break
    else:
        if str(i).find('Building Hours') > 0:
            flushing_sched[days_of_week[count]] = []
            count = count + 1
        if str(i).find('Lap Swim') > 0:
            flushing_sched[days_of_week[count-1]].append(re.findall('[0-9]*:[0-9]*\s[a-z]\s-\s[0-9]*:[0-9]*\s[a-z]'  ,str(i)))
pickle.dump( flushing_sched, open("flushing_sched.p", "wb" ) )

In [234]:
flushing_sched

{'Friday': [['6:00 a - 9:30 p']],
 'Monday': [['6:00 a - 9:30 p']],
 'Saturday': [['9:00 a - 9:30 p']],
 'Sunday': [['9:00 a - 7:30 p']],
 'Thursday': [['6:00 a - 9:30 p']],
 'Tuesday': [['6:00 a - 9:30 p']],
 'Wednesday': [['6:00 a - 9:30 p']]}

In [236]:

#8, Flushing Meadows Corona Park is just Lap Swim
#9, Roy Wilkins is Adult and Senior Swim
#10, Saint Marys is Adult Swim

r = urllib.urlopen(urls[9]).read()
soup = BeautifulSoup(r, "lxml")
other = ["program", "center-hrs"]
letters = soup.find_all(["p", "div"], other ) 

swimposting = []
otherposting = []
for i in letters:
    if str(i).find('Adult and Senior Swim') > 0:
        swimposting.append(str(i))
    if str(i).find('Building Hours') > 0:
        swimposting.append(str(i))
    else:
        otherposting.append(str(i))

roywilkins = swimposting[7:] 
roywilkins_sched = {}
count = 0
for i in roywilkins:
    if count == 8:
        break
    else:
        if str(i).find('Building Hours') > 0:
            roywilkins_sched[days_of_week[count]] = []
            count = count + 1
        if str(i).find('Adult and Senior Swim') > 0:
            roywilkins_sched[days_of_week[count-1]].append(re.findall('[0-9]*:[0-9]*\s[a-z]\s-\s[0-9]*:[0-9]*\s[a-z]'  ,str(i)))

            
pickle.dump( roywilkins_sched, open("roywilkins_sched.p", "wb" ) )


In [237]:
roywilkins_sched

{'Friday': [['8:00 a - 9:30 a'], ['5:30 p - 8:30 p']],
 'Monday': [['8:00 a - 9:30 a'], ['6:00 p - 8:30 p']],
 'Saturday': [],
 'Sunday': [],
 'Thursday': [['8:00 a - 9:30 a'], ['7:00 p - 8:30 p']],
 'Tuesday': [['8:00 a - 9:30 a'], ['7:00 p - 8:30 p']],
 'Wednesday': [['8:00 a - 9:30 a']]}

In [240]:

#8, Flushing Meadows Corona Park is just Lap Swim
#9, Roy Wilkins is Adult and Senior Swim
#10, Saint Marys is Adult Swim

r = urllib.urlopen(urls[10]).read()
soup = BeautifulSoup(r, "lxml")
other = ["program", "center-hrs"]
letters = soup.find_all(["p", "div"], other ) 

swimposting = []
otherposting = []
for i in letters:
    if str(i).find('Adult Swim') > 0:
        swimposting.append(str(i))
    if str(i).find('Building Hours') > 0:
        swimposting.append(str(i))
    else:
        otherposting.append(str(i))
saintmarys = swimposting[7:] 
saintmarys_sched = {}
count = 0
for i in saintmarys:
    if count == 7:
        break
    else:
        if str(i).find('Building Hours') > 0:
            saintmarys_sched[days_of_week[count]] = []
            count = count + 1
        if str(i).find('Adult Swim') > 0:
            saintmarys_sched[days_of_week[count-1]].append(re.findall('[0-9]*:[0-9]*\s[a-z]\s-\s[0-9]*:[0-9]*\s[a-z]'  ,str(i)))
pickle.dump( saintmarys_sched, open("saintmarys_sched.p", "wb" ) )

In [243]:
import os.path
folder = os.path.join('//', 'Users', 'cedarwaxwing', 'data_projects', '4.11.16findmypool', 'pickles_of_schedule')
for filename in os.listdir (folder):
    print filename
#try to make a loop to iterate through and turn each into a dataframe
tempnames =['Asser_Levy', 'Brownsville', 'Chelsea', 'Flushing_Meadows_Corona_Park', 'Gertrude_Eredel', 'Hansborough', 'Metropolitan',
              'Recreation_Center_54', 'Roy_Wilkins', 'Saint_Marys', 'St._Johns', 'Tony_Dapolito']
officalname = ['Asser Levy', 'Brownsville', 'Chelsea', 'Flushing Meadows Corona Park', 'Gertrude Eredel', 'Hansborough', 'Metropolitan',
              'Recreation Center 54', 'Roy Wilkins', 'Saint Marys', 'St. Johns', 'Tony Dapolito']

.DS_Store
asser_levy_sched.p
brownsville_sched.p
chelsea_sched.p
flushing_sched.p
gertrude_sched.p
hansborough_sched.p
metropolitan_sched.p
reccenter54_sched.p
roywilkins_sched.p
saintmarys_sched.p
stjohns_sched.p
tony_dapolito_sched.p


In [244]:
#brownsville = pickle.load(open("pickles_of_schedule/brownsville_sched.p", "rb"))
officalname = ['Asser Levy', 'Brownsville', 'Chelsea', 'Flushing Meadows Corona Park', 'Gertrude Eredel', 'Hansborough', 'Metropolitan','Recreation Center 54', 'Roy Wilkins', 'Saint Marys', 'St. Johns', 'Tony Dapolito']

df = pd.DataFrame()  
for j, filename in enumerate(os.listdir (folder)):
    pool = filename[:-8]
    
    pool = pickle.load(open("pickles_of_schedule/" + filename, "rb"))
    #print pool
    pool = pd.DataFrame.from_dict(pool, orient='index')
    #print pool
    pool = pool.transpose()
    pool['Pool'] = officalname[j]
    #print pool
    df = df.append(pool)



In [246]:
df
pickle.dump( df, open("pooldataframedraft.p", "wb" ) )

In [248]:
#allpools['Monday'].iloc[1][0] removing list containing times, just want the times
f = lambda x: 'None' if x==None else x[0]
df['Monday']= df['Monday'].map(f)

In [249]:
df['Tuesday'] = df['Tuesday'].map(f)
df['Wednesday'] = df['Wednesday'].map(f)
df['Thursday'] = df['Thursday'].map(f)
df['Friday'] = df['Friday'].map(f)
df['Saturday'] = df['Saturday'].map(f)
df['Sunday'] = df['Sunday'].map(f)

In [251]:
pickle.dump( df, open("pooldataframedraft.p", "wb" ) )

In [252]:
df.to_csv('pooldatafinalgood.csv')

In [254]:
con = mdb.connect('localhost', 'root', '', 'lap_schedule')

In [262]:
con = mdb.connect('localhost', 'root', '', 'lap_schedule') 
#pooldb = df.to_sql('lap_schedule_table5', con, flavor='mysql')
pooldata = pd.read_sql('SELECT * FROM lap_schedule_table5', con)


In [265]:
df.head()

,Monday,Tuesday,Friday,Wednesday,Thursday,Sunday,Saturday,Pool
0,7:00 a - 9:00 a,7:00 a - 9:00 a,7:00 a - 9:00 a,7:00 a - 9:00 a,7:00 a - 9:00 a,8:00 a - 11:00 a,8:00 a - 9:00 a,Asser Levy
1,1:15 p - 3:45 p,1:15 p - 3:45 p,1:15 p - 3:00 p,1:15 p - 3:45 p,1:15 p - 3:30 p,2:15 p - 4:30 p,2:15 p - 4:30 p,Asser Levy
2,6:30 p - 9:20 p,6:30 p - 9:20 p,7:00 p - 9:20 p,6:30 p - 9:20 p,6:30 p - 9:20 p,None,None,Asser Levy
0,2:15 p - 3:00 p,None,2:15 p - 3:00 p,2:15 p - 3:00 p,None,None,None,Brownsville
1,5:15 p - 7:00 p,None,5:15 p - 7:00 p,5:15 p - 7:00 p,None,None,None,Brownsville
